In [6]:
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import time
from typing import List

In [3]:
def get_page_soup(url_link):
    response = requests.get(url_link)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    return soup

In [4]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 1

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

In [5]:
def get_book_links(page_soup) -> List[str]:
    links = page_soup.find_all('li', attrs={'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    links = [main_url + link.a.get('href') for link in links]
    return links

In [7]:
book_links = []

for i in tqdm(np.arange(1, 51)):
    soup = get_page_soup(main_url + f'page-{i}.html')
    links = get_book_links(soup)
    book_links.extend(links)

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
def get_book_info(link):
    soup = get_page_soup(link)
    name = soup.find('h1').text
    desc = soup.find_all('p')[3].text
    if len(desc) > 10:
        desc = 1
    else:
        desc = 0
    r = soup.find_all('p')[2].get('class')[1]
    if r == 'Five':
        r = 5.0
    elif r == 'Four':
        r = 4.0
    elif r == 'Three':
        r = 3.0
    elif r == 'Two':
        r = 2.0
    else:
        r = 1.0
    
    pbt = float(soup.find_all('td')[2].text[1:])
    pat = float(soup.find_all('td')[3].text[1:])
    tax = float(soup.find_all('td')[4].text[1:])
    data = [name, desc, r, pbt, pat, tax]
    return data

In [19]:
df = pd.DataFrame(columns = ['Name', 'Description', 'Rating', 'Price_Before_TAX', 'Price_After_TAX', 'TAX'])
df

,Name,Description,Rating,Price_Before_TAX,Price_After_TAX,TAX


In [21]:
get_book_info(book_links[1])

['Tipping the Velvet', 1, 1.0, 53.74, 53.74, 0.0]

In [22]:
for link in tqdm(book_links):
    df.loc[len(df)] = get_book_info(link)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
df

,Name,Description,Rating,Price_Before_TAX,Price_After_TAX,TAX
0,A Light in the Attic,1,3.0,51.77,51.77,0.0
1,Tipping the Velvet,1,1.0,53.74,53.74,0.0
2,Soumission,1,1.0,50.10,50.10,0.0
3,Sharp Objects,1,4.0,47.82,47.82,0.0
4,Sapiens: A Brief History of Humankind,1,5.0,54.23,54.23,0.0
...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,0,1.0,55.53,55.53,0.0
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",1,4.0,57.06,57.06,0.0
997,A Spy's Devotion (The Regency Spies of London #1),1,5.0,16.97,16.97,0.0
998,1st to Die (Women's Murder Club #1),1,1.0,53.98,53.98,0.0


In [25]:
df.to_csv('books.csv')